## Importing the Dataset

In [1]:
#upload the credentials of the kaggle account
from google.colab import files
files.upload()
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
# download the dataset
!kaggle datasets download -d paultimothymooney/blood-cells
!ls

Saving kaggle.json to kaggle.json
 84% 89.0M/106M [00:00<00:00, 136MB/s]
100% 106M/106M [00:00<00:00, 167MB/s] 
blood-cells.zip  kaggle.json  sample_data


## Extracting Dataset

In [0]:
import zipfile
with zipfile.ZipFile('blood-cells.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile('dataset2-master.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

## Importing Libraries

In [0]:
import os
from tqdm import tqdm
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline

## Data Preprocessing

In [0]:
b_dir = 'dataset2-master/images/'
def load_images(folder):
  dir = b_dir + folder
  images = []
  for folders in tqdm(os.listdir(dir)):
    for files in os.listdir(os.path.join(os.path.join(dir,folders))):
      filepath = os.path.join(os.path.join(dir,folders),files)
      img = cv2.imread(filepath)
      img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      img = cv2.resize(img,(32,24))
      img = img/255.
      images.append((img,folders))
  return images

In [6]:
train = load_images('TRAIN')
test = load_images('TEST')
val = load_images('TEST_SIMPLE')

100%|██████████| 4/4 [00:00<00:00, 53.78it/s]


In [0]:
def create_dataset(data):
  x = []
  y = []
  for obj in data:
    x.append(obj[0])
    y.append(obj[1])
  x = np.array(x)
  y = np.array(y)
  return x,y

In [0]:
x_train,y_train = create_dataset(train)
x_test,y_test = create_dataset(test)
x_val,y_val = create_dataset(val)
del train
del val
del test

## Label Encoding

In [0]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_enc_train = encoder.fit_transform(y_train)
y_enc_test = encoder.transform(y_test)
y_enc_val = encoder.transform(y_val)

## One Hot Encoding

In [10]:
from keras.utils import to_categorical
y_cat_train = to_categorical(y_enc_train,4)
y_cat_test = to_categorical(y_enc_test,4)
y_cat_val = to_categorical(y_enc_val,4)

Using TensorFlow backend.


## Model Generation

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

In [0]:
batch_size = 128
epochs = 200

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(24,32,3),strides=2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

In [18]:
model.fit(x_train,y_cat_train,batch_size,epochs ,validation_data=(x_val,y_cat_val))

Train on 9957 samples, validate on 71 samples
Epoch 1/200
9957/9957 [==============================] - 1s 136us/step - loss: 1.3889 - acc: 0.2521 - val_loss: 1.3879 - val_acc: 0.1831
Epoch 2/200
9957/9957 [==============================] - 1s 92us/step - loss: 1.3866 - acc: 0.2595 - val_loss: 1.3812 - val_acc: 0.5211
Epoch 3/200
9957/9957 [==============================] - 1s 92us/step - loss: 1.3781 - acc: 0.2869 - val_loss: 1.3847 - val_acc: 0.3099
Epoch 4/200
9957/9957 [==============================] - 1s 93us/step - loss: 1.3001 - acc: 0.3753 - val_loss: 1.2834 - val_acc: 0.3099
Epoch 5/200
9957/9957 [==============================] - 1s 92us/step - loss: 1.1372 - acc: 0.4810 - val_loss: 1.1259 - val_acc: 0.5211
Epoch 6/200
9957/9957 [==============================] - 1s 92us/step - loss: 1.0246 - acc: 0.5335 - val_loss: 1.1153 - val_acc: 0.5352
Epoch 7/200
9957/9957 [==============================] - 1s 92us/step - loss: 0.9407 - acc: 0.5804 - val_loss: 0.9285 - val_acc: 0.6056
E

In [0]:
y_pred = model.predict_classes(x_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_enc_test))

              precision    recall  f1-score   support

           0       0.70      0.65      0.68       669
           1       0.90      0.94      0.92       596
           2       0.73      0.94      0.82       479
           3       0.69      0.58      0.63       743

   micro avg       0.76      0.76      0.76      2487
   macro avg       0.76      0.78      0.76      2487
weighted avg       0.75      0.76      0.75      2487



In [21]:
loss,accuracy = model.evaluate(x_test,y_cat_test)
print("Loss - ",loss)
print("Accuracy - ",accuracy)

2487/2487 [==============================] - 0s 98us/step
Loss -  1.1973469674131887
Accuracy -  0.7567350219472326
